In [1]:
from ast import literal_eval
import pandas as pd
import os
import zipfile
import shutil
from tqdm import tqdm

In [2]:
df = pd.read_csv('XML_filtered(21_30).csv', converters={'box':literal_eval})
df

,root_folder,sub_folder,xml_name,file_name,occluded,classes,box
0,Bbox_24_new,Bbox_1990,P1017_06.xml,ZED3_KSC_054721_L_P013751.png,0,person,"[1182.95, 329.9, 1246.6, 532.52]"
1,Bbox_24_new,Bbox_1990,P1017_06.xml,ZED3_KSC_054722_L_P013752.png,0,person,"[1077.96, 354.75, 1170.2, 610.82]"
2,Bbox_24_new,Bbox_1990,P1017_06.xml,ZED3_KSC_054738_L_P013762.png,1,person,"[1646.62, 567.75, 1694.86, 687.2]"
3,Bbox_24_new,Bbox_1990,P1017_06.xml,ZED3_KSC_054740_L_P013764.png,1,person,"[1512.19, 530.31, 1547.41, 607.7]"
4,Bbox_24_new,Bbox_1990,P1017_06.xml,ZED3_KSC_054741_L_P013765.png,1,stroller,"[1335.8, 567.25, 1396.33, 653.3]"
...,...,...,...,...,...,...,...
104910,Bbox_28_new,Bbox_2340,P1030_26.xml,ZED2_KSC_103515_L_P031298.png,1,person,"[716.78, 0, 895, 454.3]"
104911,Bbox_28_new,Bbox_2340,P1030_26.xml,ZED2_KSC_103515_L_P031298.png,1,car,"[0, 142.15, 340.6, 369.8]"
104912,Bbox_28_new,Bbox_2340,P1030_26.xml,ZED2_KSC_103521_L_P031299.png,0,person,"[988.24, 175.93, 1043, 326.6]"
104913,Bbox_28_new,Bbox_2340,P1030_26.xml,ZED2_KSC_103524_L_P031300.png,0,person,"[1036.79, 167.54, 1106.9, 344.5]"


In [3]:
image_name_list = df[['root_folder', 'sub_folder', 'file_name']]
image_name_list = image_name_list.drop_duplicates()
image_name_list

,root_folder,sub_folder,file_name
0,Bbox_24_new,Bbox_1990,ZED3_KSC_054721_L_P013751.png
1,Bbox_24_new,Bbox_1990,ZED3_KSC_054722_L_P013752.png
2,Bbox_24_new,Bbox_1990,ZED3_KSC_054738_L_P013762.png
3,Bbox_24_new,Bbox_1990,ZED3_KSC_054740_L_P013764.png
4,Bbox_24_new,Bbox_1990,ZED3_KSC_054741_L_P013765.png
...,...,...,...
104904,Bbox_28_new,Bbox_2340,ZED2_KSC_103513_L_P031296.png
104905,Bbox_28_new,Bbox_2340,ZED2_KSC_103514_L_P031297.png
104907,Bbox_28_new,Bbox_2340,ZED2_KSC_103515_L_P031298.png
104912,Bbox_28_new,Bbox_2340,ZED2_KSC_103521_L_P031299.png


In [4]:
zip_name_list = image_name_list['root_folder'].drop_duplicates().to_list()
zip_name_list

['Bbox_24_new',
 'Bbox_26_new',
 'Bbox_22_new',
 'Bbox_27_new',
 'Bbox_29_new',
 'Bbox_25_new',
 'Bbox_21_new',
 'Bbox_30_new',
 'Bbox_23_new',
 'Bbox_28_new']

In [5]:
# zip file 이 존재하는 ROOT DIR
# 받아야 하는 것 zip file name, dataframe
ZIP_ROOT_DIR = '/Users/sm/Downloads/인도보행 영상/바운딩박스'
EXTRACT_DIR = os.path.join(ZIP_ROOT_DIR, 'extract')
zip_dir = os.path.join(ZIP_ROOT_DIR, zip_name_list[0]+'.zip')

assert zip_dir.endswith('.zip')

#mkdir
if not os.path.exists(os.path.join(ZIP_ROOT_DIR, EXTRACT_DIR)):
    os.makedirs(os.path.join(ZIP_ROOT_DIR, EXTRACT_DIR))

condition = image_name_list['root_folder'] == zip_name_list[0]
extract_list = []

for image in image_name_list[condition].to_dict('records'):
    extract_list.append(
        os.path.join(image['sub_folder'], image['file_name'])
    )

# print(extract_list)
with zipfile.ZipFile(zip_dir, 'r') as target_zip:
    # extract_list에 있는 파일(폴더)이 이미 존재 할 경우에는 추출 안함
    print('image extract')
    for image in tqdm(extract_list):
        target_zip.extract(image, os.path.join(ZIP_ROOT_DIR, EXTRACT_DIR))
    
# 경로에 압축을 푼 이미지를 한 경로로 모은다.
subfolder_list = [i for i in os.listdir(EXTRACT_DIR) if os.path.isdir(os.path.join(EXTRACT_DIR, i))]
print()
print(f'move images to {EXTRACT_DIR}')
for folder in tqdm(subfolder_list):
    image_list = [i for i in os.listdir(os.path.join(EXTRACT_DIR, folder)) if i.endswith('.png')]
    # sub folder 마다 존재하는 이미지들을 EXTRACT_DIR 로 옮김
    for image in image_list:
        shutil.move(os.path.join(EXTRACT_DIR, folder, image), 
                    os.path.join(EXTRACT_DIR, image)
        )
    # 이미지를 다 옮긴 폴더는 삭제(경로 안에 파일이 없어야함)
    os.rmdir(os.path.join(EXTRACT_DIR, folder))
print()
print('zip start')
# 저장이 안되는 오류가 있다.
with zipfile.ZipFile(f'extract_{zip_name_list[0]}.zip', 'w') as save_zip:
    for i in tqdm(os.listdir(EXTRACT_DIR)):
        save_zip.write(os.path.join(EXTRACT_DIR, i), os.path.relpath(os.path.join(EXTRACT_DIR,i)), compress_type=zipfile.ZIP_DEFLATED)


# def extract_images()

image extract
100%|██████████| 3256/3256 [01:02<00:00, 51.82it/s]

move images to /Users/sm/Downloads/인도보행 영상/바운딩박스/extract
100%|██████████| 80/80 [00:01<00:00, 61.83it/s]

zip start
100%|██████████| 3257/3257 [05:05<00:00, 10.66it/s]


In [ ]:
# 검증용
